# FEC: 2018 election results

#### Import Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import us

In [3]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

---

## Read data

#### Get spreadsheet with results from FEC

In [4]:
sheet_name = "Table 4. Senate by Party"

In [5]:
src = pd.read_excel(
    "https://www.fec.gov/documents/2706/federalelections2018.xlsx",
    sheet_name=sheet_name,
    header=0,
    skiprows=3,
    skipfooter=17,
    names=[
        "name",
        "dem_primary",
        "gop_primary",
        "other_primary",
        "dem_votes",
        "gop_votes",
        "other_votes",
    ],
).fillna(0)

In [6]:
postal_to_name = us.states.mapping("abbr", "name")
src["state"] = src["name"].map(postal_to_name)

In [7]:
src["total_votes"] = src["dem_votes"] + src["gop_votes"] + src["other_votes"]

In [8]:
src["gop_vote_pct"] = ((src["gop_votes"] / src["total_votes"]) * 100).round(2)
src["dem_vote_pct"] = ((src["dem_votes"] / src["total_votes"]) * 100).round(2)
src["other_vote_pct"] = ((src["other_votes"] / src["total_votes"]) * 100).round(2)

In [9]:
df = (
    src[~src["gop_vote_pct"].isna()]
    .drop(
        [
            "name",
            "dem_primary",
            "gop_primary",
            "other_primary",
            "total_votes",
            "dem_votes",
            "gop_votes",
            "other_votes",
        ],
        axis=1,
    )
    .copy()
)

In [10]:
df["winner"] = (
    df[["gop_vote_pct", "dem_vote_pct", "other_vote_pct"]]
    .idxmax(axis=1)
    .str.replace("_vote_pct", "")
)

In [11]:
df.head()

,state,gop_vote_pct,dem_vote_pct,other_vote_pct,winner
3,Arizona,47.61,49.96,2.43,dem
5,California,0.00,100.00,0.00,dem
7,Connecticut,39.35,56.80,3.85,dem
8,Delaware,37.82,59.95,2.23,dem
10,Florida,50.05,49.93,0.01,gop


In [12]:
df["year"] = "2018"

In [15]:
df["gop_vote_margin"] = df["gop_vote_pct"] - df["dem_vote_pct"]
df["dem_vote_margin"] = df["dem_vote_pct"] - df["gop_vote_pct"]

In [16]:
df.to_csv("data/processed/2018_election_results_states_fec.csv", index=False)